# [1667. Fix Names in a Table](https://leetcode.com/problems/fix-names-in-a-table/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Users

<pre>+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| user_id        | int     |
| name           | varchar |
+----------------+---------+</pre>
user_id is the primary key (column with unique values) for this table.
This table contains the ID and the name of the user. The name consists of only lowercase and uppercase characters.


Write a solution to fix the names so that only the first character is uppercase and the rest are lowercase.

Return the result table ordered by user_id.

The result format is in the following example.



Example 1:

Input:
Users table:
<pre>+---------+-------+
| user_id | name  |
+---------+-------+
| 1       | aLice |
| 2       | bOB   |
+---------+-------+</pre>
Output:
<pre>+---------+-------+
| user_id | name  |
+---------+-------+
| 1       | Alice |
| 2       | Bob   |
+---------+-------+</pre>

In [2]:
# pandas schema

import pandas as pd

data = [[1, 'aLice'], [2, 'bOB']]
users = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id':'Int64', 'name':'object'})

# converting to spark dataframe

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

users_df = spark.createDataFrame(users)
users_df.show(truncate=False)

+-------+-----+
|user_id|name |
+-------+-----+
|1      |aLice|
|2      |bOB  |
+-------+-----+



In [7]:
# solving in spark dataframe

users_df.withColumn('name',initcap('name'))\
    .orderBy('user_id')\
    .show()

+-------+-----+
|user_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



In [36]:
users_df\
      .withColumn('name',
                  concat(
                    upper(substring(col('name'), 1, 1)),
                    lower(substring(col('name'), 2, 1000))
                    ))\
      .show()

+-------+-----+
|user_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



In [35]:
users_df\
      .withColumn('name',expr('concat( upper(substr(name,1,1)), lower(substr(name,2)))'))\
      .show()

+-------+-----+
|user_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



In [3]:
# solving in spark SQL

users_df.createOrReplaceTempView('users')

spark.sql('''
select
    user_id,
    initcap(name) as name
from users
order by user_id;
''').show()

+-------+-----+
|user_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



In [6]:
### mysql equivalent:
spark.sql(
'''
select
    user_id,
    concat( upper(substr(name,1,1)), lower(substr(name,2)))  as name
from users
order by user_id
'''
).show()

+-------+-----+
|user_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



In [ ]:
spark.stop()